In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

### Below is the general workflow of this project but feel free to add more additional steps as you wish and you could also not follow this workflow and designing your own!

In [2]:
#loading dataa
train = pd.read_csv('../input/cx-kaggle-final-project-fall-2021/train_datalabel.csv')
test = pd.read_csv('../input/cx-kaggle-final-project-fall-2021/test_data.csv')
submission = pd.read_csv('../input/cx-kaggle-final-project-fall-2021/sample_submission.csv')

In [3]:
#Check if there exists N/A values in the dataframes here.
train[train.isna().any(axis=1)] #157 null values out of 4088 (3%), all in BMI 
test[test.isna().any(axis=1)] #44 null values out of 1022 (4%), all in BMI

#Think about how you could handing missing values.

#Modify the dataframe here as you see fit.
#Remove null values, they're only 3% and 4% of the data

#train = train.dropna()
#train = train.reset_index(drop=True)

#test = test.dropna()
#test = test.reset_index(drop=True)


#coming back from the end. I need 1022 values for test. So I can't really drop. I'kk just grab the average BMI
#and replace it (there's not much difference between genders either)
train['bmi'].fillna(28.93,inplace=True)
test['bmi'].fillna(28.74,inplace=True)

In [4]:
#Since there are categorical variables here, do can apply one hot encoding to categorical columns!
#You can also use other techniques too if you wish.
#REMEMBER TO APPLY THE SAME TRANSFORMATION TO BOTH TRAINING AND TESTDATA.

def one_hot_encode(df):
    
    #rural = 0, urban =1
    df = df.replace("Urban", 1)
    df = df.replace("Rural", 0)
    
    #married = 1, not_married =0
    df = df.replace("Yes", 1)
    df = df.replace("No", 0)
    
    #female = 1, male =0
    df = df.replace("Female", 1)
    df = df.replace("Male", 0)
    
    #smoking - make 4 columns for formerly smoked, never smoked, unknown, smokes
    SS = df[["smoking_status"]]
    
    #1 is formerly smoked
    former = SS.replace("formerly smoked", 1)
    former = former.replace(["never smoked", "smokes", "Unknown"],0)
    df = df.assign(formerly_smoked = former.values)
    
    #2 is Never smoked
    never = SS.replace("never smoked", 1)
    never = never.replace(["formerly smoked", "smokes", "Unknown"],0)
    df = df.assign(never_smoked = never.values)
    
    #3 is smokes
    smokes = SS.replace("smokes", 1)
    smokes = smokes.replace(["never smoked", "formerly smoked", "Unknown"],0)
    df = df.assign(smokes = smokes.values)
    
    #4 is unkown 
    unknown = SS.replace("Unknown", 1)
    unknown = unknown.replace(["never smoked", "smokes", "formerly smoked"],0)
    df = df.assign(unknown_smoking = unknown.values)
    
    #remove the smoking_status column
    df = df.drop('smoking_status', axis='columns')

    
    
    #now moving onto work type Private, Self-employed, children, Govt_job, Never_worked
    WT = df[["work_type"]]
    
    #1 is Private
    Private = WT.replace("Private", 1)
    Private = Private.replace(["Self-employed", "children", "Govt_job", "Never_worked"],0)
    df = df.assign(Private_Work = Private.values)
    
    #2 is Self-employed
    SE = WT.replace("Self-employed", 1)
    SE = SE.replace(["Private", "children", "Govt_job", "Never_worked"],0)
    df = df.assign(Self_Employed = SE.values)
    
    #3 is children
    Child = WT.replace("children", 1)
    Child = Private.replace(["Self-employed", "Private", "Govt_job", "Never_worked"],0)
    df = df.assign(Children = Child.values)
    
    #4 is Govt_job
    Gov = WT.replace("Govt_job", 1)
    Gov = Gov.replace(["Self-employed", "Private", "children", "Never_worked"],0)
    df = df.assign(Government_Job = Gov.values)
    
    #5 is Never_worked
    NW = WT.replace("Private", 1)
    NW = NW.replace(["Self-employed", "children", "Govt_job", "Private"],0)
    df = df.assign(Never_Worked = NW.values)
    
    #remove the work_type column
    df = df.drop('work_type', axis='columns')
    
    
    return df
train = one_hot_encode(train)
test = one_hot_encode(test)

In [5]:
#Now all your columns are numerical, think about if you need all columns for your model.
#Drop/keep all columns as you see fit and explain your reasoning using comments.

#remove the columns related to work. I don't htink your job has to do with Strokes unless you work in
#a mine or something which is not mentioned in the job descriptions so job types are basically useless
train = train.drop(['Private_Work','Self_Employed','Children','Government_Job', 'Never_Worked'], axis='columns')
test = test.drop(['Private_Work','Self_Employed','Children','Government_Job', 'Never_Worked'], axis='columns')

#marriage also has littled to do with stroke i believe
#marriage doesn't change your heart condition unless its like you married someone whose unhealthy as well
#but thats not indicated in our marriage stats so that should be dropped
train = train.drop(['ever_married'], axis='columns')
test = test.drop(['ever_married'], axis='columns')

#while residency type may seem like htere's not much to do with stroke, urban areas have more smog
#and there have been studies that show that smog often leads to weaker hearts

train = train.drop(['Residence_type'], axis='columns')
test = test.drop(['Residence_type'], axis='columns')

#remaining columns all have to do with heart and lungs and health which are all important factors to stroke
#predictions and such

train = train.drop(['gender'], axis='columns')
test = test.drop(['gender'], axis='columns')





In [6]:
#Optinal: Scale your numerical columns as you wish. Some models perform better with scaled features.
#there is one "Other" gender. I am goign to drop.
#train = train.drop(train.index[train.loc[:,"gender"]=="Other"])
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import seaborn as sns
 
std_scaler = MinMaxScaler()
train_scaled= std_scaler.fit_transform(train.to_numpy())
train_scaled = pd.DataFrame(train_scaled, columns=["id","age","hypertension","heart_disease",
                                                   "avg_glucose_level","bmi","stroke","formerly_smoked",
                                                   "never_smoked","smokes","unknown_smoking"])


test_scaled= std_scaler.fit_transform(test.to_numpy())
test_scaled = pd.DataFrame(test_scaled, columns=["id","age","hypertension","heart_disease",
                                                   "avg_glucose_level","bmi","formerly_smoked",
                                                "never_smoked","smokes","unknown_smoking"])
test = test_scaled
train = train_scaled

In [7]:
#somehow. the y in my first KFold is all 0 the first time I've been doing it, so I'm going to shuffle my train database
train = train.sample(frac=1)

In [8]:
#Setup kfold cross validation here. Choose a k value as you see fit.

#according to this website: https://www.statology.org/k-fold-cross-validation/, i'm going to choose 5

#https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html

#y is whether or not they had a stroke
#x is all columns except stroke data
#i would want to run my model through this for loop

from sklearn.model_selection import KFold

p = 0
while p<1:
    train = train.sample(frac=1)
    kf = KFold(n_splits=5)
    X = np.array(train.drop(['stroke'], axis='columns'))
    y = train['stroke'].values
    i=0
    for train_index, val_index in kf.split(X): #i know i could just loop the regression thing itself but i got sus results the first time so...
        if i==0:
            X_train = X[train_index]
            X_val = X[val_index]
            y_train = y[train_index]
            if np.sum(y_train) == 0:
                p = 0
            y_val = y[val_index]
            i = i+1
        elif i==1:
            X_train_1 = X[train_index]
            X_val_1 = X[val_index]
            y_train_1 = y[train_index]
            if np.sum(y_train_1) == 0:
                p = 0
            y_val_1 = y[val_index]
            i = i+1
        elif i==2:
            X_train_2 = X[train_index]
            X_val_2 = X[val_index]
            y_train_2 = y[train_index]
            if np.sum(y_train_2) == 0:
                p = 0
            y_val_2 = y[val_index]
            i = i+1
        elif i==3:
            X_train_3 = X[train_index]
            X_val_3 = X[val_index]
            y_train_3 = y[train_index]
            if np.sum(y_train_3) == 0:
                p = 0
            y_val_3 = y[val_index]
            i = i+1
        else:
            X_train_4 = X[train_index]
            X_val_4 = X[val_index]
            y_train_4 = y[train_index]
            if np.sum(y_train_4) == 0:
                p = 0
            y_val_4 = y[val_index]
            i = i+1
        p=1




In [9]:
#Choose and create a model here. 
#Train the model using training data
import random
from sklearn import metrics
from sklearn.model_selection import train_test_split, GridSearchCV, KFold, cross_val_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import Ridge, Lasso, ElasticNet, LogisticRegression
from sklearn.metrics import confusion_matrix

lr = LogisticRegression()
lr = LogisticRegression(solver='liblinear')

#1st time round
print("1st set")
log_reg = lr.fit(X_train, y_train) #fits our model based on the data
y_val_pred = log_reg.predict(X_val)
cnf_matrix = metrics.confusion_matrix(y_val, y_val_pred)
print("CNF Matrix: ")
print(cnf_matrix)
print("LR Score: ")
print(lr.score(X_val, y_val))

#2nd time round
print("2nd set")
log_reg_1 = lr.fit(X_train_1, y_train_1) #fits our model based on the data
y_val_pred_1 = log_reg.predict(X_val_1)
cnf_matrix_1 = metrics.confusion_matrix(y_val_1, y_val_pred_1)
print("CNF Matrix: ")
print(cnf_matrix_1)
print("LR Score: ")
print(lr.score(X_val_1, y_val_1))

#3rd time round
print("3rd set")
log_reg_2 = lr.fit(X_train_2, y_train_2) #fits our model based on the data
y_val_pred_2 = log_reg.predict(X_val_2)
cnf_matrix_2 = metrics.confusion_matrix(y_val_2, y_val_pred_2)
print("CNF Matrix: ")
print(cnf_matrix_2)
print("LR Score: ")
print(lr.score(X_val_2, y_val_2))

#4th time round
print("4th set")
log_reg_3 = lr.fit(X_train_3, y_train_3) #fits our model based on the data
y_val_pred_3 = log_reg.predict(X_val_3)
cnf_matrix_3 = metrics.confusion_matrix(y_val_3, y_val_pred_3)
print("CNF Matrix: ")
print(cnf_matrix_3)
print("LR Score: ")
print(lr.score(X_val_3, y_val_3))

#5th time round
print("5th set")
log_reg_4 = lr.fit(X_train_4, y_train_4) #fits our model based on the data
y_val_pred_4 = log_reg.predict(X_val_4)
cnf_matrix_4 = metrics.confusion_matrix(y_val_4, y_val_pred_4)
print("CNF Matrix: ")
print(cnf_matrix_4)
print("LR Score: ")
print(lr.score(X_val_4, y_val_4))

#after doing all that copying pasting i've come to realize that i get the same sus results

1st set
CNF Matrix: 
[[770   0]
 [ 48   0]]
LR Score: 
0.941320293398533
2nd set
CNF Matrix: 
[[781   0]
 [ 37   0]]
LR Score: 
0.9547677261613692
3rd set
CNF Matrix: 
[[771   0]
 [ 47   0]]
LR Score: 
0.9425427872860636
4th set
CNF Matrix: 
[[791   0]
 [ 26   0]]
LR Score: 
0.9681762545899633
5th set
CNF Matrix: 
[[775   0]
 [ 42   0]]
LR Score: 
0.9485924112607099


In [10]:
#Measure the performance of your model on the validation set and report the error by printing out the loss.


In [11]:
#Predict test data labels using your model here. 
#Your output should be of length 1022
print("Test Data")
test_predict = log_reg.predict(test)


Test Data


In [12]:
#Change the stroke column of submission variable I definied above to your test predictions.
submission['stroke'] = test_predict
submission

,id,stroke
0,0,0.0
1,1,0.0
2,2,0.0
3,3,0.0
4,4,0.0
...,...,...
1017,1017,0.0
1018,1018,0.0
1019,1019,0.0
1020,1020,0.0


In [13]:
#Saving the csv file
#You can download this file by going to the output folder on the right side of your screen. The csv is in /kaggle/working
submission.to_csv('submission.csv', index=False)